In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from pathlib import Path

from au2v.analyst import Analyst
from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer
from au2v.model import load_model

In [ ]:
trainer_config = TrainerConfig(
    dataset_name="toydata-paper",
    epochs=3,
    ignore_saved_model=True,
    load_model=False,
    batch_size=64,
    model_dir="cache/model/",
    load_dataset=False,
)
model_config = ModelConfig(
    d_model=32
)